In [9]:
import json
import pandas as pd
from collections import defaultdict
from google.colab import files
import io

def carregar_dados():
    """Carrega os arquivos JSON necessários para a análise via upload no Google Colab"""

    print("📁 UPLOAD DOS ARQUIVOS JSON")
    print("=" * 50)
    print("Por favor, faça upload dos seguintes arquivos:")
    print("1. opinion_survey_users.json")
    print("2. opinion_survey_responses.json")
    print("3. metric-groups.json")
    print("\n🔄 Iniciando upload...")

    # Upload dos arquivos
    uploaded = files.upload()

    print(f"\n📋 Arquivos recebidos: {list(uploaded.keys())}")

    # Função para encontrar arquivo por padrão de nome
    def encontrar_arquivo(pattern, uploaded_files):
        for filename in uploaded_files.keys():
            if pattern in filename.lower():
                return filename
        return None

    # Buscar arquivos pelos padrões de nome (ignorando numeração do Colab)
    users_file = encontrar_arquivo('opinion_survey_users', uploaded)
    responses_file = encontrar_arquivo('opinion_survey_responses', uploaded)
    metrics_file = encontrar_arquivo('metric-groups', uploaded) or encontrar_arquivo('metric_groups', uploaded)

    # Verificar se todos foram encontrados
    missing = []
    if not users_file:
        missing.append('opinion_survey_users.json')
    if not responses_file:
        missing.append('opinion_survey_responses.json')
    if not metrics_file:
        missing.append('metric-groups.json')

    if missing:
        print(f"❌ Arquivos não encontrados: {missing}")
        print("📋 Arquivos disponíveis:", list(uploaded.keys()))
        raise FileNotFoundError(f"Arquivos faltando: {missing}")

    print("✅ Todos os arquivos foram identificados com sucesso!")
    print(f"✓ Usuários: {users_file}")
    print(f"✓ Respostas: {responses_file}")
    print(f"✓ Métricas: {metrics_file}")
    print("\n📊 Processando dados...")

    # Carregar e processar cada arquivo
    try:
        # Carregar dados dos usuários
        users_content = uploaded[users_file]
        users = json.loads(users_content.decode('utf-8'))
        print(f"✓ Usuários carregados: {len(users)} registros")

        # Carregar respostas da pesquisa
        responses_content = uploaded[responses_file]
        responses = json.loads(responses_content.decode('utf-8'))
        print(f"✓ Respostas carregadas: {len(responses)} registros")

        # Carregar grupos de métricas
        metrics_content = uploaded[metrics_file]
        metric_groups = json.loads(metrics_content.decode('utf-8'))
        print(f"✓ Grupos de métricas carregados: {len(metric_groups)} grupos")

        print("\n🎉 Dados carregados com sucesso!\n")

        return users, responses, metric_groups

    except json.JSONDecodeError as e:
        raise ValueError(f"❌ Erro ao decodificar JSON: {e}")
    except Exception as e:
        raise Exception(f"❌ Erro inesperado ao carregar dados: {e}")

def criar_mapeamentos(metric_groups):
    """Cria mapeamentos necessários para análise"""

    # Mapeamento ID -> Nome da métrica
    metric_id_to_name = {}
    metric_id_to_dimension = {}
    metric_id_to_indicator = {}

    for index, group in enumerate(metric_groups):
        metric_id = index + 1  # IDs começam em 1
        metric_id_to_name[metric_id] = group['Nome']
        metric_id_to_dimension[metric_id] = group['Tipo']
        metric_id_to_indicator[metric_id] = group['Classificação']

    return metric_id_to_name, metric_id_to_dimension, metric_id_to_indicator

def analisar_selecoes_metricas(responses, metric_id_to_name, metric_id_to_dimension, metric_id_to_indicator):
    """Analisa as seleções de métricas pelos especialistas"""

    # Contadores
    metric_selection_count = defaultdict(int)
    dimension_counts = defaultdict(int)
    indicator_counts = defaultdict(int)

    total_responses = len(responses)

    # Processar cada resposta
    for response in responses:
        try:
            # Parse das métricas selecionadas
            selected_metrics = json.loads(response['value'])

            for metric_id in selected_metrics:
                metric_id = int(metric_id)

                # Contar seleção da métrica
                if metric_id in metric_id_to_name:
                    metric_selection_count[metric_id] += 1

                    # Contar por dimensão
                    dimension = metric_id_to_dimension[metric_id]
                    dimension_counts[dimension] += 1

                    # Contar por indicador
                    indicator = metric_id_to_indicator[metric_id]
                    indicator_counts[indicator] += 1

        except (json.JSONDecodeError, ValueError, KeyError) as e:
            print(f"Erro ao processar resposta {response.get('id', 'unknown')}: {e}")
            continue

    return metric_selection_count, dimension_counts, indicator_counts, total_responses

def criar_ranking_metricas(metric_selection_count, metric_id_to_name, total_responses):
    """Cria ranking das métricas por frequência de seleção"""

    ranking = []

    for metric_id, count in metric_selection_count.items():
        name = metric_id_to_name[metric_id]
        percentage = (count / total_responses) * 100

        ranking.append({
            'posicao': 0,  # Será preenchido após ordenação
            'id': metric_id,
            'nome_metrica': name,
            'frequencia': count,
            'percentual': round(percentage, 2)
        })

    # Ordenar por frequência (decrescente)
    ranking.sort(key=lambda x: x['frequencia'], reverse=True)

    # Adicionar posições
    for i, item in enumerate(ranking, 1):
        item['posicao'] = i

    return ranking

def main():
    """Função principal que executa toda a análise"""

    print("🚀 ANÁLISE DA PESQUISA DE OPINIÃO")
    print("=" * 60)

    try:
        # 1. Carregar dados via upload
        users, responses, metric_groups = carregar_dados()

        # 2. Criar mapeamentos
        print("🔄 Criando mapeamentos...")
        metric_id_to_name, metric_id_to_dimension, metric_id_to_indicator = criar_mapeamentos(metric_groups)
        print("✅ Mapeamentos criados\n")

        # 3. Analisar seleções
        print("🔍 Analisando seleções de métricas...")
        metric_counts, dimension_counts, indicator_counts, total_responses = analisar_selecoes_metricas(
            responses, metric_id_to_name, metric_id_to_dimension, metric_id_to_indicator
        )
        print("✅ Análise concluída\n")

        # 4. Criar ranking
        print("📊 Criando ranking de métricas...")
        ranking = criar_ranking_metricas(metric_counts, metric_id_to_name, total_responses)
        print("✅ Ranking criado\n")

        # 5. Criar DataFrames
        print("📋 Preparando resultados...")

        # DataFrame do ranking completo
        df_ranking = pd.DataFrame(ranking)

        # DataFrame de dimensões
        total_dimensions = sum(dimension_counts.values())
        df_dimensions = pd.DataFrame([
            {
                'dimensao': dim,
                'frequencia': count,
                'percentual': round((count/total_dimensions)*100, 2)
            }
            for dim, count in dimension_counts.items()
        ]).sort_values('frequencia', ascending=False)

        # DataFrame de indicadores
        total_indicators = sum(indicator_counts.values())
        df_indicators = pd.DataFrame([
            {
                'indicador': ind,
                'frequencia': count,
                'percentual': round((count/total_indicators)*100, 2)
            }
            for ind, count in indicator_counts.items()
        ]).sort_values('frequencia', ascending=False)

        print("✅ DataFrames criados\n")

        # 6. Exibir resultados
        print("=" * 60)
        print("📈 RESULTADOS DA ANÁLISE")
        print("=" * 60)

        print(f"\n📊 RESUMO GERAL")
        print(f"Total de respostas válidas: {total_responses}")
        print(f"Total de seleções analisadas: {sum(metric_counts.values())}")

        print(f"\n🏆 RANKING DE MÉTRICAS POR FREQUÊNCIA:")
        print(df_ranking[['posicao', 'nome_metrica', 'frequencia', 'percentual']].to_string(index=False))

        print(f"\n📈 DISTRIBUIÇÃO POR DIMENSÕES:")
        print(df_dimensions.to_string(index=False))

        print(f"\n📋 DISTRIBUIÇÃO POR INDICADORES:")
        print(df_indicators.to_string(index=False))

        # Dados específicos para dissertação
        print(f"\n" + "=" * 60)
        print("🎯 DADOS PARA DISSERTAÇÃO")
        print("=" * 60)

        contributors_metric = ranking[0]
        social_data = df_dimensions[df_dimensions['dimensao'] == 'Social'].iloc[0]
        robustez_data = df_indicators[df_indicators['indicador'] == 'Robustez'].iloc[0]

        print(f"\n🎯 MÉTRICA PRIORITÁRIA:")
        print(f"Contributors and Active Developers: {contributors_metric['frequencia']} seleções ({contributors_metric['percentual']}%)")

        print(f"\n🤝 DIMENSÃO SOCIAL:")
        print(f"Social: {social_data['frequencia']} seleções ({social_data['percentual']}%)")

        print(f"\n🛡️ INDICADOR ROBUSTEZ:")
        print(f"Robustez: {robustez_data['frequencia']} seleções ({robustez_data['percentual']}%)")

        return {
            'ranking_completo': df_ranking,
            'distribuicao_dimensoes': df_dimensions,
            'distribuicao_indicadores': df_indicators,
            'resumo': {
                'total_respostas': total_responses,
                'total_selecoes': sum(metric_counts.values()),
                'metrica_prioritaria': contributors_metric,
                'dimensao_social_pct': social_data['percentual'],
                'robustez_pct': robustez_data['percentual']
            }
        }

    except Exception as e:
        print(f"❌ ERRO DURANTE A ANÁLISE: {e}")
        print("\n💡 Dicas para resolver:")
        print("- Verifique se os arquivos JSON estão no formato correto")
        print("- Certifique-se de que enviou todos os 3 arquivos necessários")
        print("- Tente executar novamente")
        return None

# Executar análise
if __name__ == "__main__":
    # Instrução de uso
    print("🎯 COMO USAR NO GOOGLE COLAB:")
    print("1. Execute esta célula")
    print("2. Faça upload dos 3 arquivos JSON quando solicitado")
    print("3. Aguarde o processamento")
    print("4. Visualize os resultados\n")

    resultados = main()

    if resultados is not None:
        # Exemplos de como acessar os dados
        print(f"\n" + "=" * 60)
        print("📚 COMO USAR OS RESULTADOS")
        print("=" * 60)
        print("\n# Para acessar o ranking completo:")
        print("ranking = resultados['ranking_completo']")
        print("\n# Para acessar distribuição por dimensões:")
        print("dimensoes = resultados['distribuicao_dimensoes']")
        print("\n# Para acessar top 5 métricas:")
        print("top5 = resultados['ranking_completo'].head(5)")
        print("\n# Para ver apenas métricas sociais:")
        print("# (você precisará cruzar com metric-groups.json)")
        print("\n# Para exportar resultados:")
        print("# resultados['ranking_completo'].to_csv('ranking_metricas.csv', index=False)")
        print("\n✅ Análise concluída com sucesso!")
    else:
        print("\n❌ Análise não pôde ser concluída. Tente novamente.")

🎯 COMO USAR NO GOOGLE COLAB:
1. Execute esta célula
2. Faça upload dos 3 arquivos JSON quando solicitado
3. Aguarde o processamento
4. Visualize os resultados

🚀 ANÁLISE DA PESQUISA DE OPINIÃO
📁 UPLOAD DOS ARQUIVOS JSON
Por favor, faça upload dos seguintes arquivos:
1. opinion_survey_users.json
2. opinion_survey_responses.json
3. metric-groups.json

🔄 Iniciando upload...


Saving opinion_survey_responses.json to opinion_survey_responses (6).json
Saving opinion_survey_users.json to opinion_survey_users (6).json
Saving metric-groups.json to metric-groups (5).json

📋 Arquivos recebidos: ['opinion_survey_responses (6).json', 'opinion_survey_users (6).json', 'metric-groups (5).json']
✅ Todos os arquivos foram identificados com sucesso!
✓ Usuários: opinion_survey_users (6).json
✓ Respostas: opinion_survey_responses (6).json
✓ Métricas: metric-groups (5).json

📊 Processando dados...
✓ Usuários carregados: 24 registros
✓ Respostas carregadas: 316 registros
✓ Grupos de métricas carregados: 47 grupos

🎉 Dados carregados com sucesso!

🔄 Criando mapeamentos...
✅ Mapeamentos criados

🔍 Analisando seleções de métricas...
✅ Análise concluída

📊 Criando ranking de métricas...
✅ Ranking criado

📋 Preparando resultados...
✅ DataFrames criados

📈 RESULTADOS DA ANÁLISE

📊 RESUMO GERAL
Total de respostas válidas: 316
Total de seleções analisadas: 1973

🏆 RANKING DE MÉTRICAS 